In [27]:
from scipy import stats
import matplotlib.pyplot as plt
import numpy as np 
import seaborn as sns
import random
import pandas as pd

In [28]:
class point:    
    def __init__(self, value):
        self.value = value
        
    def getValue(self):
        return self.value

In [139]:
class cluster:
    def __init__(self, points=[]):
        self.points = points
        self.mean = self.getMean()
        self.std = self.getStd()
        
    def values(self):
        return [p.getValue() for p in self.points]
        
    def push(self, point):
        self.points.append(point)
        self.reEval()
    
    def pop(self, point):
        self.points.remove(point)
        self.reEval()
        return point
    
    def contains(self, point):
        return point in self.points
    
    def reEval(self):
        self.mean = self.getMean()
        self.std = self.getStd()
    
    def getMean(self):
        return np.mean(self.values())
    
    def getStd(self):
        return np.std(self.values())
    
    def prob(self, point):
        return stats.norm(self.getMean(), self.getStd()).pdf(point.getValue())
    
    def numPoints(self):
        return len(self.points)
    
    def crpProb(self, point):
        print(self.std)
        return self.prob(point)*self.numPoints()

In [140]:
class crpgmm:
    def __init__(self, points, alpha):
        self.alpha = alpha
        self.clusters = []
        self.points = []
        
        for x in points:
            self.points.append(point(x))
              
    def findCluster(self, point=None):
        for i in range(len(self.clusters)):
            if self.clusters[i].contains(point):
                return self.clusters[i]     
            
    def clusterPoint(self, point):
        c = self.findCluster(point)
        
        if c is not None:
            c.pop(point)
        
        p = [_c.crpProb(point) for _c in self.clusters]
        p.append(self.alpha)
        p = np.array(p)
        p = p/np.sum(p)
        
        c = np.random.choice(len(self.clusters)+1, 1, p=p)[0]
        
        
        if c == len(self.clusters):
            self.clusters.append(cluster(points=[point]))            
        else:
            self.clusters[c].push(point)
    
    def clusterRound(self):
        for point in self.points:
            self.clusterPoint(point)
            
    def cluster(self, rounds):
        for i in range(rounds):
            self.clusterRound()

In [141]:
model = crpgmm([3, 4, 5, 3, 10, 11, 12, 13, 11], 2000)
model.cluster(1)

0.0
0.5
0.5
0.0
0.4714045207910317
0.0
2.9154759474226504
0.0
3.54400902933387
0.0
3.890872509976251
0.0
4.140393356054125
0.0


/Users/armaangoel/.local/share/virtualenvs/Learning-iB_qMzuY/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1652: RuntimeWarning: divide by zero encountered in double_scalars
  x = np.asarray((x - loc)/scale, dtype=dtyp)
/Users/armaangoel/.local/share/virtualenvs/Learning-iB_qMzuY/lib/python3.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in less


In [120]:
model.clusters[0].points

In [121]:
model.clusters[1].points